<a href="https://colab.research.google.com/github/twyeh/university-physics/blob/main/pymgrid%E7%AF%84%E4%BE%8B01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pymgrid

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 22.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 40.5 MB/s eta 0:00:00
  Created wheel for pymgrid: filename=pymgrid-1.2.2-py3-none-any.whl size=3492845 sha256=999544f3396c867d3df57828e17057419b2b5c9d4e4d884300304b719724db69
  Stored in directory: /root/.cache/pip/wheels/aa/9c/34/d880763dcc693c67448303756a6e1adb899bdc0d5080643030
Successfully built pymgrid


<img src="https://r2cdn.perplexity.ai/pplx-full-logo-primary-dark%402x.png" class="logo" width="120"/>

# 使用pymgrid 計算一個百萬瓦的發電站及10萬用戶的電路系統

pymgrid是一個專為微電網（microgrid）模擬設計的Python套件，主要用於小規模系統的研究與控制算法開發。然而，要模擬1百萬瓦（1MW）發電站與10萬用戶的電路系統，需注意以下關鍵步驟與限制：

---

### 1. **系統規模的實現限制**

- **模組容量限制**：pymgrid的預設模組（如`GensetModule`、`BassModule`）主要針對小型微電網設計，單一模組的發電/儲能容量通常為數十至數百千瓦[^1_1][^1_4]。
- **擴展方法**：
    - **並聯多個模組**：例如，用10個`GensetModule`（每個100kW）組成1MW發電站[^1_1]。
    - **自定義參數**：直接調整模組的`running_max_production`等參數，例如：

In [28]:
import numpy as np
from pymgrid import Microgrid
from pymgrid.modules import GensetModule, BatteryModule, LoadModule, RenewableModule


In [29]:
genset = GensetModule(
    running_min_production=10,
    running_max_production=50,
    genset_cost=0.5)  # 1MW

---

### 2. **10萬用戶負載模擬**

- **負載數據生成**：使用`LoadModule`時需提供時間序列數據，可通過統計方法生成：
    - **人均負載估算**：假設每戶平均功率1kW，總負載為100MW，需生成對應的時間序列：

In [31]:
import numpy
total_load = 50 * np.random.rand(100)  # 模擬一年每小時數據
load_module = LoadModule(time_series=total_load)

---

### 3. **儲能與電網整合**

- **儲能系統配置**：使用`BatteryModule`時需匹配負載需求：

In [32]:
battery = BatteryModule(
    min_capacity=0,
    max_capacity=100,  # 500MWh儲能
    max_charge=50,
    max_discharge=50,     # 1MW放電功率
    efficiency=0.9,
    init_soc=0.5
)

- **主電網交互**：通過`GridModule`設定輸電限制：

---

### 4. **控制策略範例**

- **強化學習（RL）應用**：pymgrid支援將微電網建模為馬可夫決策過程（MDP），可訓練RL代理程序優化調度[^1_2][^1_3]：

In [33]:
# Using random data
renewable = RenewableModule(time_series=50*np.random.rand(100))

load = LoadModule(time_series=60*np.random.rand(100))

microgrid = Microgrid([genset, battery, ("pv", renewable), load])

In [36]:
microgrid

Microgrid([load x 1, pv x 1, balancing x 1, genset x 1, battery x 1])

In [35]:
for j in range(10):
  action = microgrid.sample_action(strict_bound=True)
  microgrid.run(action)

TypeError: Unable to normalize scalar value, expected array-like of shape 2

In [ ]:
microgrid.get_log(drop_singleton_key=True)

---

### 5. **效能與擴展性挑戰**

- **計算資源**：10萬用戶的負載數據（8760時間點）需約3.5GB記憶體（100,000×8760×4 bytes），建議使用分布式計算或數據壓縮[^1_3]。
- **模擬速度**：大規模系統可能需簡化模型（如降低時間解析度）或使用預計算的基準場景（如`pymgrid25`）[^1_2][^1_3]。

---

### 總結

雖然pymgrid原生設計針對小型微電網，但通過模組並聯、參數調整與分區建模，仍可實現大規模系統的模擬。關鍵在於合理分配發電/儲能容量、生成代表性負載數據，並選擇適當的控制策略（如MPC或RL）來優化運行成本[^1_3][^1_5]。

<div style="text-align: center">⁂</div>

[^1_1]: https://github.com/Total-RD/pymgrid

[^1_2]: https://xinliu.engineering.ucdavis.edu/sites/g/files/dgvnsk9831/files/inline-files/pymgrid-neurips-2020.pdf

[^1_3]: https://s3.us-east-1.amazonaws.com/climate-change-ai/papers/neurips2020/3/paper.pdf

[^1_4]: https://pymgrid.readthedocs.io/en/stable/reference/api/microgrid/pymgrid.Microgrid.html

[^1_5]: https://dergipark.org.tr/en/download/article-file/1649221

[^1_6]: https://pymgrid.readthedocs.io/en/latest/examples/quick-start.html

[^1_7]: https://nips.cc/virtual/2020/20370

[^1_8]: https://pypi.org/project/microgrids/

[^1_9]: https://github.com/bhushan318/pymgridbhu

[^1_10]: https://readthedocs.org/projects/pymgrid/

[^1_11]: https://www.mathworks.com/solutions/electrification/generation-transmission-distribution.html

[^1_12]: https://juejin.cn/post/7167635797317779487

[^1_13]: https://github.com/Total-RD/pymgrid/issues/73

[^1_14]: https://ar5iv.labs.arxiv.org/html/2011.08004

[^1_15]: https://pymgrid.readthedocs.io/en/latest/examples/index.html

[^1_16]: https://paperswithcode.com/paper/pymgrid-an-open-source-python-microgrid/review/

[^1_17]: https://www.climatechange.ai/papers/neurips2020/3

[^1_18]: https://arxiv.org/abs/2011.08004

[^1_19]: https://repositorio.uniandes.edu.co/server/api/core/bitstreams/7338fb3b-5136-4c50-a109-fa5a2f0f61a6/content

[^1_20]: https://s3.us-east-1.amazonaws.com/climate-change-ai/papers/neurips2020/3/paper.pdf

[^1_21]: https://www.youtube.com/watch?v=4YQyoAie2fc

[^1_22]: https://project.linuxfoundation.org/hubfs/LF Research/Open Source Opportunity for Microgrids - Report.pdf?hsLang=en

[^1_23]: https://repositorio.uniandes.edu.co/bitstreams/7338fb3b-5136-4c50-a109-fa5a2f0f61a6/download

[^1_24]: https://pymgrid.readthedocs.io

[^1_25]: https://www.scribd.com/document/421379211/188-2486-2-PB

[^1_26]: https://pubmed.ncbi.nlm.nih.gov/39753622/

[^1_27]: https://pymgrid.readthedocs.io/en/latest/reference/microgrid.html

[^1_28]: https://www.scirp.org/pdf/sgre_2023103015514345.pdf

[^1_29]: https://paperswithcode.com/paper/pymgrid-an-open-source-python-microgrid

[^1_30]: https://pymgrid.readthedocs.io/en/stable/reference/api/microgrid/pymgrid.Microgrid.html

[^1_31]: https://riuma.uma.es/xmlui/bitstream/handle/10630/18339/Paredes_Parrilla_Angel_TFG_GIE.pdf;jsessionid=E0EF58D7561BE16A28A54F8AE3BD019E?sequence=1

[^1_32]: https://www.scirp.org/journal/paperinformation?paperid=128777